In [4]:
from collections import defaultdict
import math
import numpy as np
from numpy import ma
import datetime, time
from datetime import datetime
import netCDF4
from netCDF4 import Dataset, num2date
from scipy.interpolate import griddata

import matplotlib.pyplot as plt
plt.style.use('ggplot')
% matplotlib inline

import pickle
import halem
import datetime, time
import numpy as np
timeQ = time

"disable functions below if cartopy is unavailable"
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import halem
import halem.Mesh_maker as Mesh_maker
import halem.Flow_class as Flow_class
import os

import utide
from pandas import DataFrame

from pandas import date_range
from IPython.display import clear_output

def Tidal_analysis(time_orig, time, WD_raw, u_raw, v_raw, Rayleigh_min):
    coef_WD = utide.solve(time_orig, WD_raw,
                         lat=53,
                         nodal=False,
                         trend=False,
                         method='ols',
                         conf_int='linear',
                         Rayleigh_min=Rayleigh_min,)

    WD_predict = utide.reconstruct(time, coef_WD)


    coef_u = utide.solve(time_orig, u_raw,
                         lat=53,
                         nodal=False,
                         trend=False,
                         method='ols',
                         conf_int='linear',
                         Rayleigh_min=Rayleigh_min,)

    u_predict = utide.reconstruct(time, coef_u)

    coef_v = utide.solve(time_orig, v_raw,
                         lat=53,
                         nodal=False,
                         trend=False,
                         method='ols',
                         conf_int='linear',
                         Rayleigh_min=Rayleigh_min,)

    v_predict = utide.reconstruct(time, coef_v)
    
    
    clear_output()
    return WD_predict, u_predict, v_predict

In [2]:
nodes = np.loadtxt('D:/Use_case_Schouwen/tidal_an/nodes.csv')
t = np.loadtxt('D:/Use_case_Schouwen/tidal_an/t.csv')
WD = np.loadtxt('D:/Use_case_Schouwen/tidal_an/d.csv')
u = np.loadtxt('D:/Use_case_Schouwen/tidal_an/u.csv')
v = np.loadtxt('D:/Use_case_Schouwen/tidal_an/v.csv')
add_nodes = np.loadtxt('D:/Use_case_Schouwen/tidal_an/additional_nodes.csv')[:,::-1]

add_u = griddata(nodes, np.transpose(u), (add_nodes), method='linear')
add_v = griddata(nodes, np.transpose(v), (add_nodes), method='linear')
add_WD = griddata(nodes, np.transpose(WD), (add_nodes), method='linear')

add_u = np.transpose(add_u)
add_v = np.transpose(add_v)
add_WD = np.transpose(add_WD)

u = np.concatenate((u, add_u), axis = 1)
v = np.concatenate((v, add_v), axis = 1)
WD = np.concatenate((WD, add_WD), axis = 1)
nodes = np.concatenate((nodes, add_nodes))

In [5]:
a = 47
b = 361

time_orig = (t)/60/60/24
time = time_orig[::6][a:(a+b)]

time_stamp = []
for T in time:
    time_stamp.append(datetime.datetime.fromtimestamp((T)*60*60*24))
time_stamp = np.array(time_stamp)

time_stamp_orig = []
for T in time_orig:
    time_stamp_orig.append(datetime.datetime.fromtimestamp(T*60*60*24))
    
time_stamp_orig = np.array(time_stamp_orig)

node = [51.66709617, 3.70336611]
node = halem.Calc_path.find_startstop(node, nodes)
Rayleigh_min = 2.3

WD_raw = WD[:,node]
u_raw = u[:,node]
v_raw = v[:,node]

WD_predict, u_predict, v_predict = Tidal_analysis(time_orig, time, WD_raw, u_raw, v_raw, Rayleigh_min)

TS = time_stamp[:b]
TS2 = TS - TS[0] + TS[-1]  + datetime.timedelta(seconds = 3600)
TS3 = TS2 - TS2[0] + TS2[-1] + datetime.timedelta(seconds = 3600)
TS4 = TS3 - TS3[0] + TS3[-1] + datetime.timedelta(seconds = 3600)
TS5 = TS4 - TS4[0] + TS4[-1] + datetime.timedelta(seconds = 3600)

TS = np.concatenate((TS,TS2,TS3,TS4, TS5))
WD_df = np.concatenate((WD_predict['h'][:b],WD_predict['h'][:b],WD_predict['h'][:b],WD_predict['h'][:b],WD_predict['h'][:b]))

df2 = DataFrame(np.transpose([WD_df, 
                              np.zeros(len(TS)),
                              np.zeros(len(TS)),
                              WD_df + 7.86
                             ]), 
                index = TS, 
                columns=['Tide [m]','Hm0 [m]','Tp [s]','Water depth'])
import datetime, time

df2.head()

,Tide [m],Hm0 [m],Tp [s],Water depth
2018-05-09 23:00:00,0.383292,0.0,0.0,8.243292
2018-05-10 00:00:00,-0.187545,0.0,0.0,7.672455
2018-05-10 01:00:00,-0.716728,0.0,0.0,7.143272
2018-05-10 02:00:00,-1.063555,0.0,0.0,6.796445
2018-05-10 03:00:00,-1.249519,0.0,0.0,6.610481


In [9]:
df2.to_pickle('test')

In [11]:
df = pd.read_pickle('test')

In [12]:
df.head()

,Tide [m],Hm0 [m],Tp [s],Water depth
2018-05-09 23:00:00,0.383292,0.0,0.0,8.243292
2018-05-10 00:00:00,-0.187545,0.0,0.0,7.672455
2018-05-10 01:00:00,-0.716728,0.0,0.0,7.143272
2018-05-10 02:00:00,-1.063555,0.0,0.0,6.796445
2018-05-10 03:00:00,-1.249519,0.0,0.0,6.610481
